In [1]:
from sklearn.ensemble import AdaBoostRegressor

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State


In [2]:
data = Path('./after_data')
data_list = list(data.glob('*'))
df = pd.read_csv('./after_data/df_final_18.csv')

## Data Preprocessing

# 결측치, 이상치 제거
df_drop = df.drop([3, 11, 19, 20, 27, 28]).reset_index().drop(['index'], axis=1)

feature_cols = ['slight_cnt', 'cctv_cnt', 'fire_cnt', 'car_cnt',
       'popu_num', 'popu_sx_rate', 'local_num', 'local_sx_rate',
       'foreigner_all', 'foreigner_M', 'foreigner_F', 'bhood', 'nl_cnt']
ind_col = ['crime_safety']

X = df_drop[feature_cols]
y = df_drop[ind_col]

model = AdaBoostRegressor(base_estimator=None, n_estimators=50, random_state=113)
model.fit(X,y)

C:\Users\hrd\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


AdaBoostRegressor(random_state=113)

In [3]:
text_input = dbc.FormGroup(
    [
        dbc.Label("보안등개수"),
        dbc.Input(id = 'slight_cnt'),
    ]
)

In [4]:
colors = {
    'background': '#ffffff',
    'text': '#111111'
}
title = html.Div(
        style={'backgroundColor': colors['background'], 'padding':'10px'}, 
        children=[
            html.H1(' '),
            html.H1(
                children='체감안전도 예측',
                style={'textAlign': 'center', 'color': colors['text']}),
            html.H1(' '),
            html.Div(children='FORECASTING SAFETY', 
                     style={'textAlign': 'center', 'color': colors['text']}),
            html.H1(' '),
            html.H1(' ')
        ])

In [5]:
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Page 1", href="#")),
        dbc.DropdownMenu(
            children=[
                dbc.DropdownMenuItem("More pages", header=True),
                dbc.DropdownMenuItem("Page 2", href="#"),
                dbc.DropdownMenuItem("Page 3", href="#"),
            ],
            nav=True,
            in_navbar=True,
            label="More",
        ),
    ],
    brand="Safety from the Crime",
    brand_href="#",
    color="primary",
    dark=True,
)

In [6]:
cards = html.Div(
    [
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('보안등개수'),
                    dcc.Input(id="slight_cnt"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('CCTV개수'),
                    dcc.Input(id="cctv_cnt"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('화재발생건수'),
                    dcc.Input(id="fire_cnt"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('교통사고발생건수'),
                    dcc.Input(id="car_cnt"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('인구수'),
                    dcc.Input(id="popu_num"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('성비'),
                    dcc.Input(id="popu_sx_rate"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('내국인수'),
                    dcc.Input(id="local_num"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('내국인성비'),
                    dcc.Input(id="local_sx_rate")
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('외국인수'),
                    dcc.Input(id="foreigner_all"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('남성외국인수'),
                    dcc.Input(id="foreigner_M"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('여성외국인수'),
                    dcc.Input(id="foreigner_F"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('기초수급자수'),
                    dcc.Input(id="bhood"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    html.Label('유흥업소수'),
                    dcc.Input(id="nl_cnt"),
                ]
            ),
        ),
        dbc.Card(
            dbc.CardBody(
                [
                    dbc.Button(
                        '예측시작',
                        id = 'button',
                        n_clicks = 0
                    ),
                    html.Hr(),
                    html.Div(
                        [
                            html.H6(id = 'crime_safety')
                        ]
                    )
                ]
            ),
        )
    ], style={'textAlign': 'center'}
)


In [7]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(
    [
        title,
        navbar,
        cards
    ]
)

In [8]:
@app.callback(
    Output('crime_safety','children'),
    Input('button','n_clicks'),
    State('slight_cnt','value'),
    State('cctv_cnt','value'),
    State('fire_cnt','value'),
    State('car_cnt','value'),
    State('popu_num','value'),
    State('popu_sx_rate','value'),
    State('local_num','value'),
    State('local_sx_rate','value'),
    State('foreigner_all','value'),
    State('foreigner_M','value'),
    State('foreigner_F','value'),
    State('bhood','value'),
    State('nl_cnt','value'),
)
def predict_crime_safety(n,val1,val2,val3,val4,val5,
                         val6,val7,val8,val9,val10,val11,val12,val13):
    x = [[val1,val2,val3,val4,val5,
                         val6,val7,val8,val9,val10,val11,val12,val13]]
    result = model.predict(x)
    result = str(result[0])
    return html.Div(f'안전도는 {result} 입니다.')

In [ ]:
app.run_server(port='7777')

Dash is running on http://127.0.0.1:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Aug/2021 13:16:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2021 13:16:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2021 13:16:44] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_data(func(*args

127.0.0.1 - - [06/Aug/2021 13:16:44] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [06/Aug/2021 13:17:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2021 13:20:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2021 13:20:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2021 13:20:38] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hrd\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\hrd\Anaconda3\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_data(func(*args

127.0.0.1 - - [06/Aug/2021 13:20:39] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [06/Aug/2021 15:52:50] "POST /_dash-update-component HTTP/1.1" 200 -


In [18]:
import numpy as np
np.random.seed(2021)

In [19]:
# np.random.seed(2021)
print(np.random.random())
print(np.random.random())
print(np.random.random())
print(np.random.random())
print(np.random.random())

0.6059782788074047
0.7333693611934982
0.13894715672839875
0.31267308385468695
0.9972432813403187


In [21]:
np.random.seed(2021)
print(np.random.random())
print(np.random.random())
print(np.random.random())
print(np.random.random())
print(np.random.random())

0.6059782788074047
0.7333693611934982
0.13894715672839875
0.31267308385468695
0.9972432813403187
